In [1]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas
from pandas import DataFrame
import csv

In [2]:
with open('iproperty.csv', mode='w') as csv_file:
    fieldnames = ['link', 
                  'name', 
                  'type', 
                  'price', 
                  'address', 
                  'built_up', 
                  'land_area', 
                  'bedrooms', 
                  'bathrooms', 
                  'monthly_installment', 
                  'land_title', 
                  'tenure',
                  'price_per_sqft',
                  #'maintenance_fee',
                  #'near_by_amenities',
                  #'facilities',
                  #'furnishing',
                  #'floor_level',
                  'state']
    #fieldnames = ['link','type','state']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

In [3]:
property_link = []
property_name = []
property_type = []
property_price = []
property_address = []
property_built_up = []
property_land_area = []
property_bedrooms = []
property_bathrooms = []
property_monthly_installment = []
property_land_title = []
property_tenure = []
property_price_per_sqft = []
#property_maintenance_fee = []
#property_near_by_amenities = []
#property_facilities = []
#property_furnishing = []
#property_floor_level = []
property_state = []

In [4]:
def scraping(webpage, page_number):
    next_page = webpage + str(page_number)
    response= requests.get(str(next_page))
    soup = BeautifulSoup(response.content,"html.parser")
    soup_link = soup.findAll("div", attrs={"class":'PremiumCardstyle__BottomDescriptionWrapper-iGyVff czlWoj'})
    soup_type = soup.findAll("p",{"class":"attributes-description-item ListingAttributesstyle__ListingAttrsDescriptionItemWrapper-eRPpTH gjIvWo"})
    soup_state = soup.findAll("div",{"class":"PremiumCardstyle__AddressWrapper-jSJtsE hlWmFZ"})
    #print(len(soup_link))
    #print(len(soup_state))
    
    for x in range(len(soup_link)):
        pty_link = soup_link[x].find('a', { "class": "depth-listing-card-link" })
        if pty_link.text=="View details":
            if pty_link['href'] != '/property/iskandar-puteri-nusajaya/sale-8440612/' or pty_link['href'] != '/property/johor-bahru/sale-7776353/' or pty_link['href'] != '/property/usj/sale-8366347/' or pty_link['href'] != '/property/iskandar-puteri-nusajaya/sale-8413421/' or pty_link['href'] != "/property/kl-sentral/suasana-sentral-loft/sale-5611538/" or pty_link['href'] != "/property/johor-bahru/sale-8148057/":
                property_link.append(pty_link['href'])
                property_type.append(soup_type[x].text.strip().split('•')[0])
                property_state.append(soup_state[x].text.strip().split(',')[1])

                if pty_link.text=="View details":
                    detail_page = "https://www.iproperty.com.my" + pty_link['href']
                    print(detail_page)
                    response= requests.get(str(detail_page))
                    soup = BeautifulSoup(response.content,"html.parser")
                    soup_name = soup.find("h1",attrs={"class":'PropertySummarystyle__ProjectTitleWrapper-cvJoqY ifcFYS'})
                    soup_price = soup.find("div", attrs={"class":'property-price ListingPrice__Price-cmdnCI ieWmRM'})
                    soup_address_1 = soup.findAll("span", {"class":"property-address sale-default"})
                    soup_address_2 = soup.findAll("span", {"class":"property-address new-ppp"})
                    soup_built_up_land_area = soup.findAll("li",attrs={"class":'PropertySummarystyle__AreaInfoItem-lmykjH kdRqkQ'})
                    soup_bedrooms = soup.find("ul", {"class":"property-features PropertyFeatures__PropertyFeaturesWrapper-fAoHZZ WBFTH"}) 
                    soup_bathrooms = soup.find("ul", {"class":"property-features PropertyFeatures__PropertyFeaturesWrapper-fAoHZZ WBFTH"}) 
                    soup_monthly_installment = soup.find("div", {"class":"money-per-month MortgageCalculatorstyle__MoneyWrapper-iRttZT haasVb"})
                    soup_land_title_1 = soup.findAll("div", {"class":"ListingAttributesstyle__AttributeItemData-dogytt dCAMnf"})
                    soup_land_title_2 = soup.findAll("div", {"class":"ListingAttributesstyle__AttributeItemData-dogytt hfXwpL"})
                    soup_tenure = soup.findAll("div", {"class":"ListingAttributesstyle__AttributeItemData-dogytt dCAMnf"})
                    soup_price_per_sqft_1 = soup.findAll("div", {"class":"ListingAttributesstyle__AttributeItemData-dogytt hfXwpL"})
                    soup_price_per_sqft_2 = soup.findAll("div", {"class":"ListingAttributesstyle__AttributeItemData-dogytt dCAMnf"})
                    soup_maintenance_fee_1 = soup.findAll("div", {"class":"ListingAttributesstyle__AttributeItemData-dogytt dCAMnf"})
                    soup_maintenance_fee_2 = soup.findAll("div", {"class":"ListingAttributesstyle__AttributeItemData-dogytt hfXwpL"})
                    soup_facilities = soup.find("div", {"class":"feature-description-content ListingFeatureDescription__FeatureDescriptionContent-dFVvWS hPahWk"})
                    #print(soup_bathrooms.findAll("li")[1])

                    property_name.append(soup_name.text.strip())

                    if 'Contact' in soup_price.text.strip() or 'From' in soup_price.text.strip():
                        property_price.append(0)
                    else:
                        property_price.append(soup_price.text.strip().split('RM')[1].replace(',', ''))

                    try:
                        property_address.append(soup_address_1[0].text.strip())
                    except IndexError:
                        property_address.append(soup_address_2[0].text.strip())

                    property_built_up.append(soup_built_up_land_area[0].text.strip().split(':')[1].replace(',', ''))
                    property_land_area.append(soup_built_up_land_area[0].text.strip().split(':')[1].replace(',', ''))

                    try:
                        if soup_bedrooms==None:
                            property_bedrooms.append("0")
                        else:
                            property_bedrooms.append(soup_bedrooms.findAll("li")[0].text.strip())
                    except IndexError:
                        if soup_bedrooms==None:
                            property_bedrooms.append("0")
                        else:
                            property_bedrooms.append(soup_bedrooms.findAll("li")[0].text.strip())

                    try:
                        if soup_bathrooms==None:
                            property_bathrooms.append("0")
                        else:
                            try:
                                property_bathrooms.append(soup_bathrooms.findAll("li")[1].text.strip())  
                            except IndexError:
                                property_bathrooms.append("0")
                    except IndexError:
                        if soup_bathrooms==None:
                            property_bathrooms.append("0")
                        else:
                            property_bathrooms.append(soup_bathrooms.findAll("li")[1].text.strip()) 

                    property_monthly_installment.append(soup_monthly_installment.text.strip().split('RM')[1].replace('per month', ''))

                    try:
                        property_land_title.append(soup_land_title_1[0].text.strip())
                    except IndexError:
                        property_land_title.append(soup_land_title_2[0].text.strip())

                    try:
                        property_tenure.append(soup_tenure[4].text.strip())
                    except IndexError:
                        property_tenure.append("N/A")

                    try:
                        if soup_price_per_sqft_1[1].text.strip()=="Vacant" or soup_price_per_sqft_1[1].text.strip()=="Tenanted" or soup_price_per_sqft_1[1].text.strip()=="Unknown":
                            property_price_per_sqft.append('-')
                        else:
                            property_price_per_sqft.append(soup_price_per_sqft_1[1].text.strip())
                    except IndexError:
                        if soup_price_per_sqft_2[1].text.strip()=="Vacant" or soup_price_per_sqft_2[1].text.strip()=="Tenanted" or soup_price_per_sqft_2[1].text.strip()=="Unknown":
                            property_price_per_sqft.append('-')
                        else:
                            property_price_per_sqft.append(soup_price_per_sqft_2[1].text.strip())

                    """
                    try:
                        property_maintenance_fee.append(soup_maintenance_fee_1[10].text.strip())
                    except IndexError:
                        property_maintenance_fee.append(soup_maintenance_fee_2[2].text.strip())
                    """

                    #property_facilities.append(soup_facilities.text.strip())
            
    if page_number < 100:
        page_number = page_number + 1
        scraping(webpage, page_number)

In [5]:
scraping('https://www.iproperty.com.my/sale/all-residential/?page=', 1)

https://www.iproperty.com.my/property/klcc/8-conlay/sale-8112108/
https://www.iproperty.com.my/property/klcc/the-oval/sale-7850546/
https://www.iproperty.com.my/property/jalan-ipoh/sale-5335970/
https://www.iproperty.com.my/new-property/property/sepang/setia-warisan-tropika/new-6076/
https://www.iproperty.com.my/property/bandar-utama/sale-8364427/
https://www.iproperty.com.my/property/klcc/opus-kl/sale-7443298/
https://www.iproperty.com.my/new-property/property/batu-pahat/laman-bayu/new-6074/
https://www.iproperty.com.my/property/skudai/sale-8367138/
https://www.iproperty.com.my/property/cyberjaya/sale-8234909/
https://www.iproperty.com.my/property/cyberjaya/sale-7053604/
https://www.iproperty.com.my/new-property/property/bandar-sri-damansara/ativo-suites-damansara-avenue/new-5688/
https://www.iproperty.com.my/property/cyberjaya/sale-7020841/
https://www.iproperty.com.my/property/cyberjaya/sale-8241102/
https://www.iproperty.com.my/property/cyberjaya/sale-7014295/
https://www.iproperty

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
data = { 
    'link': property_link,
    'name':property_name, 
    'type':property_type, 
    'price':property_price, 
    'address':property_address,
    'built_up':property_built_up,
    'land_area':property_land_area,
    'bedrooms':property_bedrooms,
    'bathrooms':property_bathrooms,
    'monthly_installment':property_monthly_installment,
    'land_title':property_land_title,
    'tenure':property_tenure,
    'price_per_sqft':property_price_per_sqft,
    #'maintenance_fee':property_maintenance_fee,
    #'near_by_amenities':property_near_by_amenities,
    #'facilities':property_facilities,
    #'furnishing':property_furnishing,
    #'floor_level':property_floor_level,
    'state':property_state
}

In [ ]:
print(data)

In [ ]:
print ([len(v) for v in data.values()])

In [ ]:
print(property_bedrooms)

In [ ]:
#df = DataFrame(data, columns = ['link','name','type','price','address','bedrooms','bathrooms','state'])
df = DataFrame(data, columns = ['link', 
                  'name', 
                  'type', 
                  'price', 
                  'address', 
                  'built_up', 
                  'land_area', 
                  'bedrooms', 
                  'bathrooms', 
                  'monthly_installment', 
                  'land_title', 
                  'tenure',
                  'price_per_sqft',
                  #'maintenance_fee',
                  #'near_by_amenities',
                  #'facilities',
                  #'furnishing',
                  #'floor_level',
                  'state'])
df.to_csv(r'/Users/zero/Documents/GitHub/deep_property/iproperty.csv')